In [1]:
#!pip install -qq -U diffusers==0.11.1 transformers ftfy gradio accelerate

import inspect
from typing import List, Optional, Union

import numpy as np
import torch

import PIL
import gradio as gr
from diffusers import StableDiffusionInpaintPipelineLegacy

import requests
from io import BytesIO

device = "cuda"
model_path = '/home/ec2-user/attribute_probe/mit_states_src/training_scripts/inversion/individual_token_per_image/coral/outputs_solo_texture_model'

pipe = StableDiffusionInpaintPipelineLegacy.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
).to(device)





The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.
/home/ec2-user/.local/lib/python3.9/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [2]:
import diffusers
diffusers.__file__

'/home/ec2-user/.local/lib/python3.9/site-packages/diffusers/__init__.py'

In [3]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = PIL.Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


In [4]:
import requests
from io import BytesIO

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = PIL.Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

def predict(dict, prompt):
  image =  dict['image'].convert("RGB").resize((512, 512))
  # mask_image = dict['mask'].convert("RGB").resize((512, 512))
  # mask_image.save("mask.png")
  print('USING SAVED MASK')
  mask_image = PIL.Image.open("mask.png").convert("RGB").resize((512, 512))
  images = pipe(prompt=prompt,  num_inference_steps=100, guidance_scale=12.5, image=image, mask_image=mask_image, num_images_per_prompt = 4).images
  return(images[0], images[1], images[2], images[3])

In [5]:
gr.Interface(
    predict,
    title = 'Stable Diffusion In-Painting',
    inputs=[
        gr.Image(source = 'upload', tool = 'sketch', type = 'pil'),
        gr.Textbox(label = 'prompt')
    ],
    outputs = [
        gr.Image(), gr.Image(), gr.Image(), gr.Image()
        ]
).launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a013357b72c8e5435d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


USING SAVED MASK


  0%|          | 0/80 [00:00<?, ?it/s]